In [85]:
%autosave 5

Autosaving every 5 seconds


In [86]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [87]:
import os 
import pickle
os.chdir('/content/drive/My Drive/MusicRecommendationSystem/Lalitha-Evaluation-1')
!ls
with open('weights22kwithfeatures.pickle', 'rb') as f:
    model_weights = pickle.load(f)

playlist_matrix = model_weights[0]
print(playlist_matrix.shape)
songs_matrix = model_weights[1]
print(songs_matrix.shape)

22kPlaylist+features+songindex+artistindex.csv	Evaluation-Lalitha.ipynb
Artists-weights22kwithfeatures.pickle		weights22kwithfeatures.pickle
(22740, 11)
(289796, 11)


In [88]:
''' Example code '''
import numpy as np
m = np.array([[1,2,3],[4,5,6],[7,8,9]])
c = np.array([0,1,2])
print(m * c[:, np.newaxis])

[[ 0  0  0]
 [ 4  5  6]
 [14 16 18]]


In [89]:
''' Weighting each feature 
 11 features : features = ['Danceability','Instrumentalness','Loudness','Speechiness','Valence','Energy','Mode','Tempo','Livenss','Key','Acousticness'] '''
weights=[0.5, 2, 2, 1, 1, 0.5, 0.5, 1, 0.5, 0.5, 0.75] 
songs_matrix_weighted = songs_matrix * weights[:np.newaxis]

In [90]:
def cosine_similarity(word_vectors,track_id,n):
  cosine_dict = {}
  word_list = []
  index = track_id -1 
  target = word_vectors[index,:]
  for i in range(0,word_vectors.shape[0]):
    curr = word_vectors[i]
    cos_sim = np.dot(curr,target)/(np.linalg.norm(curr)*np.linalg.norm(target))
    cosine_dict[i]=cos_sim
  dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True)
  for item in dist_sort:
    word_list.append((item[0]+1,item[1]))
  return word_list[0:n]

In [91]:
import pandas as pd
pd.set_option('display.max_columns', None)
train = pd.read_csv('22kPlaylist+features+songindex+artistindex.csv')
train.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,trackid,artist_name,track_name,duration_ms,album_name,pid,Danceability,Instrumentalness,Loudness,Speechiness,Valence,Energy,Mode,Tempo,Livenss,Key,Acousticness,Unnamed: 0.1.1.1.1,trackindex,artistindex
0,0,0,0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook,0,0.904,0.006970,-7.105,0.1210,0.810,0.813,0,125.461,0.0471,4,0.03110,NaN,18621,32614
1,1,1,1,1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,198800,In The Zone,0,0.774,0.025000,-3.914,0.1140,0.924,0.838,0,143.040,0.2420,5,0.02490,NaN,234387,6668
2,2,2,2,2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.000000,-6.583,0.2100,0.701,0.758,0,99.259,0.0598,2,0.00238,NaN,19991,5065
3,3,3,3,3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,267266,Justified,0,0.891,0.000234,-6.055,0.1400,0.818,0.714,0,100.972,0.0521,4,0.20200,NaN,43850,24706
4,4,4,4,4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,227600,Hot Shot,0,0.853,0.000000,-4.596,0.0713,0.654,0.606,1,94.759,0.3130,0,0.05610,NaN,66278,41903


In [92]:
input_song_names = ['Green Utopia', 'EASE', 'Closer']
song_ids = []
for song_name in input_song_names:
  rows = train.loc[train['track_name'] == song_name]
  try:
    song_id = rows['trackindex'].to_list()[0]
    song_ids.append(song_id)
  except:
    song_id = -1
  print(song_id)

print(song_ids) 
similar_song_ids = []
rec_ids = set()
for input_id in song_ids:
  print(input_id)
  get = cosine_similarity(songs_matrix_weighted,input_id,10)
  for rec_id in get:
    if rec_id not in rec_ids:
      rec_ids.add(rec_id)
      temp = []
      temp.append(rec_id[1])
      temp.append(rec_id[0])  
      temp.append(input_id)
      if temp[1] != temp[2]: #input song and recommended song shouldn't be the same
        similar_song_ids.append(temp)
similar_song_ids.sort()
print(similar_song_ids)

167965
13512
104585
[167965, 13512, 104585]
167965
13512
104585
[[0.9999597467317642, 128550, 104585], [0.9999597596162222, 28210, 104585], [0.9999636978410368, 232234, 104585], [0.999966134216494, 221253, 104585], [0.9999699133506669, 204742, 104585], [0.9999702269574747, 286789, 104585], [0.9999733755925552, 61960, 104585], [0.9999761146241344, 180870, 104585], [0.9999879148048764, 223366, 104585], [0.999990536993366, 195795, 167965], [0.999990973025613, 229361, 167965], [0.9999914936784996, 119885, 167965], [0.9999916505683533, 85180, 167965], [0.9999916861717626, 8821, 167965], [0.9999919154833611, 86376, 13512], [0.9999919647022002, 126510, 167965], [0.9999923957353744, 218502, 13512], [0.9999927472848091, 160918, 167965], [0.9999931593500823, 45795, 13512], [0.9999944559227681, 228196, 13512], [0.9999944709042671, 202325, 13512], [0.9999946178254447, 80166, 13512], [0.9999948695812869, 240312, 167965], [0.9999949431253627, 176103, 13512], [0.9999959563643279, 176880, 13512], [0.9

In [93]:
most_popular = ['HUMBLE', 'One Dance', 'Closer', 'Broccoli ft. Lil Yatchy', 'Congratulations', 'Bad and Boujee', 'Caroline', 'Bounce Back', 'Location', 'Mask Off', 'Despacito - Remix', "I'm the One", 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix']
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  inp_song  = inp_rows['track_name'].tolist()[0]
  rows = train.loc[train['trackindex'] == track[1]]
  song_name = rows['track_name'].tolist()
  print(song_name[0])
if len(similar_song_ids) == 0:
  for song in most_popular:
    print(song)

Ladies Don't Play Guitar
Gush
Rocío de Todos los Campos
How The Gods Kill
Worship the Devil
You're In It Now
He's Making A Fool Out Of You
God Is Good
Son Navideño
深海少女 -初音ミク「マジカルミライ 2014」Live-
A Night in Paris
Cloak and Dagger
Sax Rohmer #1
Lions
Play Me
Crackin - Martin Garrix Edit
Dali - From Portrait in Seven Shades
Maggie
Feel Me Ni
Ramblin' Man - Single Version
Whatever It Is, I Just Can't Stop
Monotonija
Jubilee Stomp
Scary Monsters And Nice Sprites - Kaskade Remix
Tu Nos Creaste
I'm Good
I'll Be Back Again


In [94]:
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  rec_rows = train.loc[train['trackindex'] == track[1]]
  # print(inp_rows.columns)
  req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness',
       'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
  inp_features = inp_rows[req_cols].values.tolist()[0]
  rec_features = rec_rows[req_cols].values.tolist()[0]
  for i in range(0, len(inp_features)):
    print(str(req_cols[i])+"\t"+str(inp_features[i])+"\t"+str(rec_features[i]))
  
  print("\n")

artist_name	Ne-Yo	Tennis
album_name	Year Of The Gentleman	Yours Conditionally
track_name	Closer	Ladies Don't Play Guitar
Danceability	0.7090000000000001	0.6759999999999999
Instrumentalness	5.2e-05	4.5e-05
Loudness	-6.437	-5.5089999999999995
Speechiness	0.0738	0.0273
Valence	0.5760000000000001	0.6609999999999999
Energy	0.745	0.557
Mode	0	0
Tempo	126.027	99.955
Livenss	0.154	0.0623
Key	4	7
Acousticness	0.0225	0.38799999999999996


artist_name	Ne-Yo	Pharrell Williams
album_name	Year Of The Gentleman	G I R L
track_name	Closer	Gush
Danceability	0.7090000000000001	0.7959999999999999
Instrumentalness	5.2e-05	1.55e-05
Loudness	-6.437	-7.062
Speechiness	0.0738	0.0537
Valence	0.5760000000000001	0.7140000000000001
Energy	0.745	0.6970000000000001
Mode	0	0
Tempo	126.027	111.999
Livenss	0.154	0.0523
Key	4	4
Acousticness	0.0225	0.112


artist_name	Ne-Yo	Natalia Lafourcade
album_name	Year Of The Gentleman	Musas
track_name	Closer	Rocío de Todos los Campos
Danceability	0.7090000000000001	0.506
Instrumen

**First Method of Evaluation**

For a given recommended song (**Sr**), find out the most similar input song  (**Si**)using cosine similarity as the metric

For 0 <= i < len(atrributes), precompute and store the ranges of each atrribute (**R_i**). Based on the ranges, define an interval **Delta_i** (the limit in which we want to call songs similar wrt that attribute)

For Sr and Si, for each attribute, calculate **diff_i = |inp_i - rec_i| / R_i**

Count the number of attributes which have diff_i <= Delta_i

If that number > 3 (or some preset threshold), then the recommmended song is considered as a **Good Recommendation**




In [95]:
attribute_ranges = []
numeric_cols = ['Danceability','Instrumentalness','Loudness','Speechiness','Valence','Energy','Mode','Tempo','Livenss','Key','Acousticness']
for attr in numeric_cols:
  print(attr)
  attr_index = train.columns.get_loc(attr)
  df_min = train[train[attr]==train[attr].min()].values.tolist()[0]
  print(df_min[attr_index])
  df_max = train[train[attr]==train[attr].max()].values.tolist()[0]
  print(df_max[attr_index])
  attribute_ranges.append(float(df_max[attr_index]-df_min[attr_index]))

print(attribute_ranges)

Danceability
0.0
0.991
Instrumentalness
0.0
0.9990000000000001
Loudness
-60.0
2.766
Speechiness
0.0
0.965
Valence
0.0
1.0
Energy
0.0
1.0
Mode
0
1
Tempo
0.0
249.703
Livenss
0.0
1.0
Key
0
11
Acousticness
0.0
0.996
[0.991, 0.9990000000000001, 62.766, 0.965, 1.0, 1.0, 1.0, 249.703, 1.0, 11.0, 0.996]


In [96]:
'''Returns True is the song is a good recommendation, False if it is not'''
def evaluate(item, attribute_ranges):
    reco = item[1]
    closest_inp = item[2]
    inp_rows = train.loc[train['trackindex'] == closest_inp]
    rec_rows = train.loc[train['trackindex'] == reco]
    # print(inp_rows.columns)
    req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness', 'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
    inp_features = inp_rows[req_cols].values.tolist()[0]
    rec_features = rec_rows[req_cols].values.tolist()[0]
    artist_index = train.columns.get_loc('artist_name')
    album_index = train.columns.get_loc('album_name')
    if inp_features[artist_index] == rec_features[artist_index]:
      return True
    if inp_features[album_index] == rec_features[album_index]:
      return True
    count = 0
    for i in range(3, len(inp_features)):
      val = float(abs(rec_features[i] - inp_features[i])/attribute_ranges[i-3])
      if val <= 0.2:
        count += 1
    if count >= 6:
      return True
    return False


count = 0
for item in similar_song_ids:
  if evaluate(item, attribute_ranges) == True:
    print("Good")
    count+=1
  else:
    print("Bad")

print(str(count) + " out of " + str(len(similar_song_ids)) + " songs are Good Recommendations")

Good
Good
Good
Good
Good
Bad
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Bad
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
25 out of 27 songs are Good Recommendations
